In [1]:
import pandas as pd

orders = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_orders_dataset.csv')
order_items = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_order_items_dataset.csv')
products = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_products_dataset.csv')
reviews = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_order_reviews_dataset.csv')
customers = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_customers_dataset.csv')
payments = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/olist_order_payments_dataset.csv')
category_translation = pd.read_csv('C:/Users/Nisarg/Desktop/oilst/product_category_name_translation.csv')

orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


# Check status counts

In [2]:
print(orders['order_status'].value_counts())

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64


# Add return flag

In [3]:
orders['is_returned'] = orders['order_status'].apply(lambda x: 1 if x == 'canceled' else 0)

# Calculate overall return rate

return_rate = orders['is_returned'].mean()
print(f"Overall Return Rate: {round(return_rate * 100, 2)}%")

Overall Return Rate: 0.63%


# Merge Orders + Order Items + Products

In [4]:
orders_items_merged = pd.merge(order_items, orders[['order_id', 'is_returned']], on='order_id', how='left')

orders_full = pd.merge(orders_items_merged, products[['product_id', 'product_category_name']], on='product_id', how='left')

category_return = orders_full.groupby('product_category_name')['is_returned'].agg(['count', 'sum'])
category_return['return_rate'] = (category_return['sum'] / category_return['count']) * 100
category_return = category_return.sort_values(by='return_rate', ascending=False)

# View top 10 categories with highest return rate

print(category_return.head(10))

                                               count  sum  return_rate
product_category_name                                                 
pc_gamer                                           9    1    11.111111
portateis_cozinha_e_preparadores_de_alimentos     15    1     6.666667
dvds_blu_ray                                      64    2     3.125000
construcao_ferramentas_seguranca                 194    5     2.577320
fraldas_higiene                                   39    1     2.564103
construcao_ferramentas_jardim                    238    4     1.680672
instrumentos_musicais                            680   11     1.617647
livros_interesse_geral                           553    7     1.265823
eletrodomesticos_2                               238    3     1.260504
eletroportateis                                  679    8     1.178203


# Return Rate by Price Range

In [5]:
# Add price range column
def price_bucket(price):
    if price < 50:
        return 'Low (< ₹50)'
    elif 50 <= price < 150:
        return 'Medium (₹50–149)'
    elif 150 <= price < 500:
        return 'High (₹150–499)'
    else:
        return 'Premium (₹500+)'

orders_full['price_range'] = orders_full['price'].apply(price_bucket)

# Return rate by price range
price_return = orders_full.groupby('price_range')['is_returned'].agg(['count', 'sum'])
price_return['return_rate'] = (price_return['sum'] / price_return['count']) * 100
price_return = price_return.sort_values(by='return_rate', ascending=False)

print(price_return)


                  count  sum  return_rate
price_range                              
Premium (₹500+)    3239   32     0.987959
High (₹150–499)   19525  125     0.640205
Low (< ₹50)       39024  178     0.456130
Medium (₹50–149)  50862  207     0.406984


In [6]:
# Select required columns
export_df = orders_full[[
    'order_id',
    'product_id',
    'product_category_name',
    'price',
    'price_range',
    'is_returned'
]]

# Export to CSV
export_df.to_csv('return_analysis_clean.csv', index=False)
